# 2023 AI/ML UnicornGym - Track 1

@jesamkim | 2023-Feb-08

# 7. 모델 빌딩 파이프라인

### setup environment

In [1]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('./src')

In [2]:
import boto3
import sagemaker
import pandas as pd

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 

In [3]:
%store

Stored variables and their in-db values:
bucket                           -> 'sagemaker-us-east-1-376278017302'
endpoint_name                    -> 'sagemaker-xgboost-2023-02-08-05-34-18-143'
hyperparameters                  -> {'scale_pos_weight': '19', 'max_depth': '2', 'eta'
image_uri                        -> '683313688378.dkr.ecr.us-east-1.amazonaws.com/sage
input_data_uri                   -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
project_prefix                   -> 'titanic-ml'
s3_path                          -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
test_data_uri                    -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
test_df                          ->     PassengerId  Pclass  Sex   Age  SibSp  Parch  
train_data_uri                   -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
train_model_artifact             -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/


---
## 7-1. 모델 빌딩 파이프라인 의 스텝(Step) 생성



### 모델 빌딩 파이프라인 변수 생성

이 단계에서는 파이프라인 변수를 사용하지 않습니다.

### 배포에 사용할 세이지 메이커 모델 정의 

실시간 엔드포인트를 생성 하기 위해 SageMaker Model을 생성합니다.

```python
model = Model(
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)
```

구체적으로, `TrainingStep` 인스턴스인 `step_train`의 속성 중 `TrainingImagea, S3ModelArtifacts` 의 결과물을 입력으로 다시 사용합니다.
- 아래의 예시는 '세이지 메이커 모델 생성' 스텝을 독립적으로 실행하기 위해서, 이전 노트북의 결과인 image_uri (모델 훈련시 다커 컨테이너 경로), model_data 또한 이전 노트북에서 모델 훈련의 아티펙트의 경로인 train_model_artifact를 사용합니다.
- [알림] image_uri 를 여기서는 모델 훈련시의 다커 컨테이너와 동일하게 추론에서 사용했습니다. 추론 다커 컨테이너를 따로 구성해서 사용할 수도 있습니다.
- [알림] `TrainingStep`의 `properties` 속성은 [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) API의 응답결과와 동일합니다.


In [4]:
from sagemaker.model import Model

model = Model(
    image_uri=image_uri,
    model_data= train_model_artifact,
    sagemaker_session=sagemaker_session,
    role=role,
)

### 세이지 메이커 모델 생성 스탭 생성

In [5]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
)
step_create_model = CreateModelStep(
    name="Survived-Basic-CreateModel",
    model=model,
    inputs=inputs,
)

---
## 7-2. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행



### 파이프라인 정의

In [6]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    steps=[step_create_model],
)

### 파이프라인 정의 확인
위에서 정의한 파이프라인 정의는 JSON 형식으로 정의 되어 있습니다.

In [7]:
import json


definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Survived-Basic-CreateModel',
   'Type': 'Model',
   'Arguments': {'ExecutionRoleArn': 'arn:aws:iam::376278017302:role/service-role/AmazonSageMaker-ExecutionRole-20230112T204234',
    'PrimaryContainer': {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1',
     'Environment': {},
     'ModelDataUrl': 's3://sagemaker-us-east-1-376278017302/titanic-ml/training_jobs/pipelines-354rt2nvtof4-Survived-Basic-Train-QHhe2K0bjJ/output/model.tar.gz'}}}]}

### 파이프라인 정의를 제출하고 실행하기
파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것 입니다.

In [8]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [9]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:376278017302:pipeline/titanic-ml',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:376278017302:pipeline/titanic-ml/execution/9k6he39ze4w4',
 'PipelineExecutionDisplayName': 'execution-1675851887353',
 'PipelineExecutionStatus': 'Succeeded',
 'PipelineExperimentConfig': {'ExperimentName': 'titanic-ml',
  'TrialName': '9k6he39ze4w4'},
 'CreationTime': datetime.datetime(2023, 2, 8, 10, 24, 47, 195000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 2, 8, 10, 24, 50, 565000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'c78999be-801b-4f92-90d7-7a53bae9f25f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c78999be-801b-4f92-90d7-7a53bae9f25f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '471',
   'date': 'Wed, 08 Feb 2023 10:24:51 GMT'},
  'RetryAttempts': 0}}

### 파이프라인 실행 기다리기

In [10]:
execution.wait()

---
## 7-3. 세이지 메이커 스튜디오에서 확인하기
- 아래의 그림 처럼 SageMaker Studio에 로긴후에 따라하시면, SageMaker Studio 에서도 실행 내역을 확인할 수 있습니다.
    - SageMaker Studio 개발자 가이드 --> [SageMaker Studio](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/studio.html)



![create_model_sm.png](img/create_model_sm.png)

---
## 7-4. 모델 아티펙트 경로 추출

### 세이지 메이커 모델의 경로를 저장함. 
- 다음 노트북에서 사용합니다.

In [11]:
response = execution.list_steps()
sagemaker_model = response[0]['Metadata']['Model']['Arn'].split("/")[-1]
print("sagemaker_model: \n", sagemaker_model)

%store sagemaker_model

sagemaker_model: 
 pipelines-9k6he39ze4w4-survived-basic-creat-txfhcvksge
Stored 'sagemaker_model' (str)
